In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/Delta_time.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

This is a good performing model

In [7]:
%%capture
!pip install tensorflow_addons

# CNN-LSTM

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN_LSTM/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN_LSTM/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [11]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 9s 54ms/step - loss: 1.4043 - accuracy: 0.5048 - val_loss: 1.3741 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 22ms/step - loss: 1.3476 - accuracy: 0.4822 - val_loss: 1.3191 - val_accuracy: 0.5151
Epoch 3/100
29/29 [==============================] - 0s 12ms/step - loss: 1.2943 - accuracy: 0.5011 - val_loss: 1.2685 - val_accuracy: 0.4741
Epoch 4/100
29/29 [==============================] - 0s 13ms/step - loss: 1.2452 - accuracy: 0.5054 - val_loss: 1.2216 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 1.2002 - accuracy: 0.5119 - val_loss: 1.1783 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 1s 21ms/step - loss: 1.1577 - accuracy: 0.5186 - val_loss: 1.1381 - val_accuracy: 0.5162
Epoch 7/100
29/29 [==============================] - 0s 13ms/step - loss: 1.1172 - accuracy: 0.5450 - val_loss: 1.1015 - val_accuracy: 0.4989
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 13s 60ms/step - loss: 1.4058 - accuracy: 0.4977 - val_loss: 1.3758 - val_accuracy: 0.5045
Epoch 2/100
28/28 [==============================] - 1s 22ms/step - loss: 1.3498 - accuracy: 0.5048 - val_loss: 1.3221 - val_accuracy: 0.5057
Epoch 3/100
28/28 [==============================] - 0s 14ms/step - loss: 1.2977 - accuracy: 0.5113 - val_loss: 1.2724 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 1.2490 - accuracy: 0.5291 - val_loss: 1.2267 - val_accuracy: 0.5068
Epoch 5/100
28/28 [==============================] - 1s 22ms/step - loss: 1.2040 - accuracy: 0.5342 - val_loss: 1.1848 - val_accuracy: 0.5170
Epoch 6/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1605 - accuracy: 0.5572 - val_loss: 1.1472 - val_accuracy: 0.4932
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1216 - accuracy: 0.5563 - val_loss: 1.1118 - val_accuracy: 0.5170
Epoch 8/100
28/28

...

Epoch 2/100
29/29 [==============================] - 1s 20ms/step - loss: 0.0309 - accuracy: 0.9900 - val_loss: 0.6786 - val_accuracy: 0.6002
Epoch 3/100
29/29 [==============================] - 0s 14ms/step - loss: 0.0199 - accuracy: 0.9960 - val_loss: 0.6645 - val_accuracy: 0.6002
Epoch 4/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0118 - accuracy: 0.9989 - val_loss: 0.6319 - val_accuracy: 0.6961
Epoch 5/100
29/29 [==============================] - 1s 28ms/step - loss: 0.0114 - accuracy: 0.9987 - val_loss: 0.5973 - val_accuracy: 0.7468
Epoch 6/100
29/29 [==============================] - 0s 17ms/step - loss: 0.0142 - accuracy: 0.9981 - val_loss: 0.5700 - val_accuracy: 0.7371
Epoch 7/100
29/29 [==============================] - 1s 25ms/step - loss: 0.0221 - accuracy: 0.9941 - val_loss: 0.5376 - val_accuracy: 0.8082
Epoch 8/100
29/29 [==============================] - 1s 30ms/step - loss: 0.0220 - accuracy: 0.9941 - val_loss: 0.5131 - val_accuracy: 0.8211
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 7s 56ms/step - loss: 0.0372 - accuracy: 0.9875 - val_loss: 0.6835 - val_accuracy: 0.5792
Epoch 2/100
28/28 [==============================] - 1s 22ms/step - loss: 0.0258 - accuracy: 0.9932 - val_loss: 0.6733 - val_accuracy: 0.5962
Epoch 3/100
28/28 [==============================] - 1s 24ms/step - loss: 0.0208 - accuracy: 0.9960 - val_loss: 0.6522 - val_accuracy: 0.6731
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0256 - accuracy: 0.9935 - val_loss: 0.6328 - val_accuracy: 0.7308
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0211 - accuracy: 0.9946 - val_loss: 0.6098 - val_accuracy: 0.7115
Epoch 6/100
28/28 [==============================] - 1s 28ms/step - loss: 0.0149 - accuracy: 0.9980 - val_loss: 0.5654 - val_accuracy: 0.7624
Epoch 7/100
28/28 [==============================] - 1s 22ms/step - loss: 0.0159 - accuracy: 0.9972 - val_loss: 0.5187 - val_accuracy: 0.8201
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 7s 56ms/step - loss: 0.0530 - accuracy: 0.9858 - val_loss: 0.6860 - val_accuracy: 0.5837
Epoch 2/100
31/31 [==============================] - 1s 22ms/step - loss: 0.0280 - accuracy: 0.9930 - val_loss: 0.6709 - val_accuracy: 0.6374
Epoch 3/100
31/31 [==============================] - 1s 22ms/step - loss: 0.0301 - accuracy: 0.9915 - val_loss: 0.6504 - val_accuracy: 0.6725
Epoch 4/100
31/31 [==============================] - 1s 23ms/step - loss: 0.0229 - accuracy: 0.9948 - val_loss: 0.6183 - val_accuracy: 0.7252
Epoch 5/100
31/31 [==============================] - 1s 23ms/step - loss: 0.0313 - accuracy: 0.9907 - val_loss: 0.5951 - val_accuracy: 0.7624
Epoch 6/100
31/31 [==============================] - 1s 26ms/step - loss: 0.0185 - accuracy: 0.9959 - val_loss: 0.5548 - val_accuracy: 0.7758
Epoch 7/100
31/31 [==============================] - 1s 26ms/step - loss: 0.0138 - accuracy: 0.9979 - val_loss: 0.5036 - val_accuracy: 0.8037
Epoch 8/100
31/31 

In [12]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.536,0.544,0.442,0.488,0.442,0.630,0.072
1,1,0.561,0.602,0.359,0.450,0.359,0.763,0.121
2,2,0.572,0.585,0.496,0.537,0.496,0.649,0.145
3,0,0.642,0.647,0.626,0.637,0.626,0.658,0.285
4,1,0.643,0.655,0.603,0.628,0.603,0.682,0.285
5,2,0.632,0.655,0.556,0.602,0.556,0.707,0.263
6,0,0.744,0.745,0.740,0.743,0.740,0.747,0.487
7,1,0.764,0.748,0.797,0.772,0.797,0.732,0.528
8,2,0.765,0.777,0.743,0.760,0.743,0.787,0.530
9,0,0.816,0.815,0.817,0.816,0.817,0.814,0.631


In [13]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_LSTM/Delta_Time_CNN_LSTM.csv', index = False)

# CNN

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [15]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 1s 18ms/step - loss: 1.7578 - accuracy: 0.5391 - val_loss: 1.7559 - val_accuracy: 0.5065
Epoch 3/100
29/29 [==============================] - 1s 19ms/step - loss: 1.7471 - accuracy: 0.5579 - val_loss: 1.7473 - val_accuracy: 0.5194
Epoch 4/100
29/29 [==============================] - 1s 20ms/step - loss: 1.7364 - accuracy: 0.5595 - val_loss: 1.7387 - val_accuracy: 0.5463
Epoch 5/100
29/29 [==============================] - 0s 11ms/step - loss: 1.7253 - accuracy: 0.5692 - val_loss: 1.7307 - val_accuracy: 0.4774
Epoch 6/100
29/29 [==============================] - 0s 12ms/step - loss: 1.7160 - accuracy: 0.5749 - val_loss: 1.7223 - val_accuracy: 0.5022
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 1.7043 - accuracy: 0.5924 - val_loss: 1.7138 - val_accuracy: 0.5269
Epoch 8/100
29/29 [==============================] - 0s 11ms/step - loss: 1.6945 - accuracy: 0.6018 - val_loss: 1.7057 - val_accuracy: 0.5216
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 18ms/step - loss: 1.7582 - accuracy: 0.5410 - val_loss: 1.7565 - val_accuracy: 0.5294
Epoch 3/100
28/28 [==============================] - 1s 18ms/step - loss: 1.7465 - accuracy: 0.5727 - val_loss: 1.7481 - val_accuracy: 0.5475
Epoch 4/100
28/28 [==============================] - 0s 11ms/step - loss: 1.7360 - accuracy: 0.5747 - val_loss: 1.7399 - val_accuracy: 0.5238
Epoch 5/100
28/28 [==============================] - 0s 12ms/step - loss: 1.7250 - accuracy: 0.5840 - val_loss: 1.7318 - val_accuracy: 0.5192
Epoch 6/100
28/28 [==============================] - 0s 11ms/step - loss: 1.7143 - accuracy: 0.5948 - val_loss: 1.7238 - val_accuracy: 0.5238
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 1.7039 - accuracy: 0.5951 - val_loss: 1.7158 - val_accuracy: 0.5475
Epoch 8/100
28/28 [==============================] - 0s 14ms/step - loss: 1.6950 - accuracy: 0.5976 - val_loss: 1.7080 - val_accuracy: 0.5226
Epoch 9/100
28/28 

...

29/29 [==============================] - 0s 16ms/step - loss: 0.4174 - accuracy: 0.9814 - val_loss: 0.9978 - val_accuracy: 0.6627
Epoch 3/100
29/29 [==============================] - 0s 11ms/step - loss: 0.4059 - accuracy: 0.9830 - val_loss: 0.9901 - val_accuracy: 0.6573
Epoch 4/100
29/29 [==============================] - 0s 10ms/step - loss: 0.4052 - accuracy: 0.9863 - val_loss: 0.9909 - val_accuracy: 0.6412
Epoch 5/100
29/29 [==============================] - 0s 11ms/step - loss: 0.4072 - accuracy: 0.9836 - val_loss: 0.9720 - val_accuracy: 0.6595
Epoch 6/100
29/29 [==============================] - 0s 16ms/step - loss: 0.3957 - accuracy: 0.9911 - val_loss: 0.9612 - val_accuracy: 0.6810
Epoch 7/100
29/29 [==============================] - 0s 10ms/step - loss: 0.3973 - accuracy: 0.9903 - val_loss: 0.9710 - val_accuracy: 0.6616
Epoch 8/100
29/29 [==============================] - 0s 10ms/step - loss: 0.3958 - accuracy: 0.9900 - val_loss: 0.9897 - val_accuracy: 0.6347
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 19ms/step - loss: 0.4116 - accuracy: 0.9827 - val_loss: 0.9950 - val_accuracy: 0.6538
Epoch 3/100
28/28 [==============================] - 0s 17ms/step - loss: 0.4013 - accuracy: 0.9890 - val_loss: 0.9851 - val_accuracy: 0.6833
Epoch 4/100
28/28 [==============================] - 0s 11ms/step - loss: 0.3960 - accuracy: 0.9890 - val_loss: 0.9785 - val_accuracy: 0.6753
Epoch 5/100
28/28 [==============================] - 0s 17ms/step - loss: 0.3952 - accuracy: 0.9926 - val_loss: 0.9690 - val_accuracy: 0.6912
Epoch 6/100
28/28 [==============================] - 0s 12ms/step - loss: 0.3934 - accuracy: 0.9918 - val_loss: 0.9704 - val_accuracy: 0.6731
Epoch 7/100
28/28 [==============================] - 0s 12ms/step - loss: 0.4030 - accuracy: 0.9847 - val_loss: 0.9586 - val_accuracy: 0.6878
Epoch 8/100
28/28 [==============================] - 1s 20ms/step - loss: 0.3922 - accuracy: 0.9935 - val_loss: 0.9437 - val_accuracy: 0.7002
Epoch 9/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 1s 16ms/step - loss: 0.4371 - accuracy: 0.9693 - val_loss: 1.0020 - val_accuracy: 0.6219
Epoch 3/100
31/31 [==============================] - 1s 18ms/step - loss: 0.4179 - accuracy: 0.9817 - val_loss: 0.9881 - val_accuracy: 0.6488
Epoch 4/100
31/31 [==============================] - 1s 21ms/step - loss: 0.4136 - accuracy: 0.9819 - val_loss: 0.9804 - val_accuracy: 0.6519
Epoch 5/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4275 - accuracy: 0.9729 - val_loss: 0.9785 - val_accuracy: 0.6446
Epoch 6/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4099 - accuracy: 0.9829 - val_loss: 0.9728 - val_accuracy: 0.6457
Epoch 7/100
31/31 [==============================] - 0s 13ms/step - loss: 0.4026 - accuracy: 0.9899 - val_loss: 0.9828 - val_accuracy: 0.6364
Epoch 8/100
31/31 [==============================] - 0s 14ms/step - loss: 0.4169 - accuracy: 0.9793 - val_loss: 0.9739 - val_accuracy: 0.6508
Epoch 9/100
31/31 

In [16]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.554,0.559,0.514,0.536,0.514,0.595,0.109
1,1,0.576,0.594,0.482,0.532,0.482,0.671,0.153
2,2,0.586,0.631,0.416,0.501,0.416,0.757,0.173
3,0,0.659,0.677,0.610,0.641,0.610,0.709,0.318
4,1,0.653,0.649,0.667,0.658,0.667,0.640,0.306
5,2,0.677,0.690,0.643,0.665,0.643,0.711,0.353
6,0,0.719,0.733,0.688,0.710,0.688,0.749,0.437
7,1,0.749,0.737,0.773,0.754,0.773,0.725,0.497
8,2,0.765,0.735,0.829,0.779,0.829,0.701,0.530
9,0,0.775,0.753,0.817,0.784,0.817,0.732,0.549


In [17]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN/Delta_time_CNN.csv', index = False)

#GRU

In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time_domain/Delta/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [19]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Train shape: (4640, 18, 29), Test shape: (1194, 18, 29)
Train shape: (4418, 18, 29), Test shape: (1416, 18, 29)
Train shape: (4838, 18, 29), Test shape: (996, 18, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 18, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 9, 256)            0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 6s 49ms/step - loss: 1.4308 - accuracy: 0.4838 - val_loss: 1.4278 - val_accuracy: 0.4838
Epoch 2/100
29/29 [==============================] - 1s 18ms/step - loss: 1.4249 - accuracy: 0.5121 - val_loss: 1.4226 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 12ms/step - loss: 1.4199 - accuracy: 0.5100 - val_loss: 1.4174 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 20ms/step - loss: 1.4145 - accuracy: 0.5094 - val_loss: 1.4122 - val_accuracy: 0.4881
Epoch 5/100
29/29 [==============================] - 1s 22ms/step - loss: 1.4092 - accuracy: 0.5202 - val_loss: 1.4070 - val_accuracy: 0.4957
Epoch 6/100
29/29 [==============================] - 1s 23ms/step - loss: 1.4035 - accuracy: 0.5353 - val_loss: 1.4019 - val_accuracy: 0.5022
Epoch 7/100
29/29 [==============================] - 1s 25ms/step - loss: 1.3986 - accuracy: 0.5318 - val_loss: 1.3968 - val_accuracy: 0.5194
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 50ms/step - loss: 1.4307 - accuracy: 0.4977 - val_loss: 1.4280 - val_accuracy: 0.5204
Epoch 2/100
28/28 [==============================] - 1s 21ms/step - loss: 1.4257 - accuracy: 0.4980 - val_loss: 1.4229 - val_accuracy: 0.5238
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.4202 - accuracy: 0.5195 - val_loss: 1.4179 - val_accuracy: 0.5215
Epoch 4/100
28/28 [==============================] - 1s 22ms/step - loss: 1.4150 - accuracy: 0.5187 - val_loss: 1.4128 - val_accuracy: 0.5294
Epoch 5/100
28/28 [==============================] - 0s 16ms/step - loss: 1.4099 - accuracy: 0.5221 - val_loss: 1.4079 - val_accuracy: 0.4853
Epoch 6/100
28/28 [==============================] - 0s 17ms/step - loss: 1.4047 - accuracy: 0.5272 - val_loss: 1.4029 - val_accuracy: 0.4842
Epoch 7/100
28/28 [==============================] - 0s 17ms/step - loss: 1.3998 - accuracy: 0.5405 - val_loss: 1.3980 - val_accuracy: 0.4955
Epoch 8/100
28/28 

...

29/29 [==============================] - 9s 52ms/step - loss: 0.3377 - accuracy: 0.9456 - val_loss: 0.8507 - val_accuracy: 0.6034
Epoch 2/100
29/29 [==============================] - 0s 15ms/step - loss: 0.3063 - accuracy: 0.9518 - val_loss: 0.8496 - val_accuracy: 0.5851
Epoch 3/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2842 - accuracy: 0.9652 - val_loss: 0.8372 - val_accuracy: 0.6185
Epoch 4/100
29/29 [==============================] - 1s 24ms/step - loss: 0.2726 - accuracy: 0.9717 - val_loss: 0.8245 - val_accuracy: 0.6573
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2751 - accuracy: 0.9696 - val_loss: 0.8194 - val_accuracy: 0.6336
Epoch 6/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2790 - accuracy: 0.9661 - val_loss: 0.8093 - val_accuracy: 0.6638
Epoch 7/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2744 - accuracy: 0.9696 - val_loss: 0.7972 - val_accuracy: 0.7026
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 74ms/step - loss: 0.3139 - accuracy: 0.9522 - val_loss: 0.8573 - val_accuracy: 0.5543
Epoch 2/100
28/28 [==============================] - 1s 31ms/step - loss: 0.3245 - accuracy: 0.9457 - val_loss: 0.8368 - val_accuracy: 0.6346
Epoch 3/100
28/28 [==============================] - 1s 25ms/step - loss: 0.2797 - accuracy: 0.9680 - val_loss: 0.8313 - val_accuracy: 0.6369
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.2758 - accuracy: 0.9677 - val_loss: 0.8211 - val_accuracy: 0.6731
Epoch 5/100
28/28 [==============================] - 0s 15ms/step - loss: 0.2713 - accuracy: 0.9643 - val_loss: 0.8190 - val_accuracy: 0.6437
Epoch 6/100
28/28 [==============================] - 1s 22ms/step - loss: 0.2713 - accuracy: 0.9689 - val_loss: 0.8032 - val_accuracy: 0.6889
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 0.2728 - accuracy: 0.9660 - val_loss: 0.8083 - val_accuracy: 0.6471
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 8s 49ms/step - loss: 0.3436 - accuracy: 0.9426 - val_loss: 0.8451 - val_accuracy: 0.6064
Epoch 2/100
31/31 [==============================] - 1s 21ms/step - loss: 0.3334 - accuracy: 0.9465 - val_loss: 0.8324 - val_accuracy: 0.6405
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.3114 - accuracy: 0.9525 - val_loss: 0.8311 - val_accuracy: 0.6198
Epoch 4/100
31/31 [==============================] - 1s 21ms/step - loss: 0.3077 - accuracy: 0.9504 - val_loss: 0.8096 - val_accuracy: 0.6880
Epoch 5/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3038 - accuracy: 0.9478 - val_loss: 0.8085 - val_accuracy: 0.6498
Epoch 6/100
31/31 [==============================] - 0s 14ms/step - loss: 0.2934 - accuracy: 0.9589 - val_loss: 0.7936 - val_accuracy: 0.6870
Epoch 7/100
31/31 [==============================] - 0s 13ms/step - loss: 0.3055 - accuracy: 0.9540 - val_loss: 0.7896 - val_accuracy: 0.6756
Epoch 8/100
31/31 

In [20]:
metrics_df_gru

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.552764,0.567452,0.443886,0.498120,0.443886,0.661642,0.105528
1,1,0.561441,0.566009,0.526836,0.545721,0.526836,0.596045,0.122881
2,2,0.558233,0.615079,0.311245,0.413333,0.311245,0.805221,0.116466
3,0,0.649079,0.643087,0.670017,0.656276,0.670017,0.628141,0.298157
4,1,0.641949,0.619217,0.737288,0.673114,0.737288,0.546610,0.283898
5,2,0.666667,0.685268,0.616466,0.649049,0.616466,0.716867,0.333333
6,0,0.714405,0.706452,0.733668,0.719803,0.733668,0.695142,0.428811
7,1,0.747175,0.733957,0.775424,0.754121,0.775424,0.718927,0.494350
8,2,0.762048,0.772443,0.742972,0.757421,0.742972,0.781124,0.524096
9,0,0.759631,0.765411,0.748744,0.756986,0.748744,0.770519,0.519263


In [21]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/Time Domain/CNN_GRU/Delta_time_GRU.csv', index = False)